## Log returns and Simple returns

Insight: If innovations in log returns are iid then, log returns at long enough horizons must be normal (Central Limit Theorem)

$$1+𝑅_{1→𝑇}=((1+𝑅_1 )(1+𝑅_2 )…(1+𝑅_𝑇))⁡\\
ln⁡(1+𝑅_{1→𝑇} )=ln⁡((1+𝑅_1 )(1+𝑅_2 )…(1+𝑅_𝑇 ))\\
ln⁡(1+𝑅_{1→𝑇} )=ln⁡(1+𝑅_1 )+ln⁡(1+𝑅_2 )+…+ln⁡(1+𝑅_𝑇 )\\
                    𝑟_{1→𝑇}=𝑟_1+𝑟_2+…+𝑟_𝑇 $$ 



* Thus, if T is large, $𝑟_{1→𝑇}$ will be approximately normally distributed. (central limit theorem)

* $𝑟_𝑡=ln⁡(1+𝑅_𝑡 )$ is the rate of return assuming continuous compounding for the period t.

* This turns out to hold up well if you look at horizons longer than a month, but does not work at all at daily frequency (much fatter left tail than the normal distribution predicts)


* Log returns are also very convenient when thinking about long-term investing

In [33]:
print([np.log(1+Data.MKT).skew(),X.skew()])

print([np.log(1+Data.MKT).kurtosis(),X.kurtosis()])

[-0.7707864524331097, -0.10212480191113225]
[2.859221702344219, -0.17043746558020745]


#### How to go back and fourth between log and simple returns?

Let R be a simple net return per period, i.e., if you invest 10 , you get 10(1+R) in the end of the period

R is a number like 5% (0.05)

We say 1+R, a number like 1.05 is a gross return. 

Why gross? Because it includes 1, your initial investment.

* To get log returns: $r=log(1+R)$


* To get back simple returns: $R=exp(r)-1$

In [34]:
r=np.log(1+Data.MKT)
R=np.exp(r)-1
pd.concat([R,Data.MKT],axis=1).head()

,MKT,MKT
Date,,
1963-02-28,-0.0215,-0.0215
1963-03-31,0.0331,0.0331
1963-04-30,0.0476,0.0476
1963-05-31,0.0200,0.0200
1963-06-30,-0.0177,-0.0177


In [35]:
threshold=3
# now real data in log returns

A=((np.log(Data.MKT+1)-np.log(Data.MKT+1).mean())<-threshold*np.log(Data.MKT+1).std())
B=((X-X.mean())<-threshold*X.std())
[A.sum(),B.sum()]

[4, 0]